In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
# Define the date range for the data
TODAY = datetime.now().date()
START = TODAY - timedelta(days=365)
# Load AAPL data
def load_data(ticker):
    data = yf.download(ticker, START, TODAY)
    data.reset_index(inplace=True)
    return data


data = load_data('AAPL')


# Preprocess the data
data = data[['Date', 'Close']]  # Use only Date and Close price
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)




# Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)


# Create training and testing datasets
training_data_len = int(np.ceil(len(scaled_data) * 0.8))  # 80% for training
train_data = scaled_data[0:training_data_len]


# Split the data into x_train and y_train
def create_dataset(data, time_step=1):
    x, y = [], []
    for i in range(len(data) - time_step - 1):
        x.append(data[i:(i + time_step), 0])
        y.append(data[i + time_step, 0])
    return np.array(x), np.array(y)
time_step = 60  # Using the last 60 days to predict the next day
x_train, y_train = create_dataset(train_data, time_step)


# Reshape the data for the GRU model
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)


# Build the GRU model
model = keras.Sequential()
model.add(layers.GRU(50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(layers.GRU(50, return_sequences=False))
model.add(layers.Dense(25))
model.add(layers.Dense(1))


# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')


# Train the model
model.fit(x_train, y_train, batch_size=1, epochs=10)


# Create the test dataset
test_data = scaled_data[training_data_len - time_step:]
x_test, y_test = create_dataset(test_data, time_step)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)


# Get predictions
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)  # Inverse scaling to original prices
# Create a DataFrame for predictions
predicted_df = pd.DataFrame(predictions, columns=['Predictions'], index=pd.date_range(start=data.index[training_data_len], periods=len(predictions), freq='B'))


# Create valid_data DataFrame for comparison
valid_data = data[training_data_len:]
valid_data['Predictions'] = predicted_df['Predictions']


# Calculate MAPE
valid_data['Error'] = valid_data['Close'] - valid_data['Predictions']
valid_data['Abs Error'] = np.abs(valid_data['Error'])
valid_data['Percentage Error'] = (valid_data['Abs Error'] / valid_data['Close']) * 100


mape = valid_data['Percentage Error'].mean()
print(f'MAPE: {mape:.2f}%')


# Plotting
plt.figure(figsize=(16, 8))
plt.title('Model Predictions vs Actual Prices')
plt.xlabel('Date')
plt.ylabel('Close Price USD')
plt.plot(train_data['Close'])
plt.plot(valid_data[['Close', 'Predictions']])
plt.legend(['Train', 'Actual', 'Predictions'], loc='lower right')
plt.show()


[*********************100%***********************]  1 of 1 completed

Epoch 1/10



c:\Users\ADMIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


139/139 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.0173
Epoch 2/10
139/139 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.0054
Epoch 3/10
139/139 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.0043
Epoch 4/10
139/139 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0054
Epoch 5/10
139/139 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.0029
Epoch 6/10
139/139 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0027
Epoch 7/10
139/139 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 0.0025
Epoch 8/10
139/139 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - loss: 0.0046
Epoch 9/10
139/139 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 0.0055
Epoch 10/10
139/139 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.0035
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 360ms/step


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_22536\852873677.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_data['Predictions'] = predicted_df['Predictions']


ValueError: Cannot set a DataFrame with multiple columns to the single column Error